<a href="https://colab.research.google.com/github/OluwajobaOluwabori/Sentiment-Analysis/blob/main/Basic_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Import neccessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd



In [ ]:
dataset = pd.read_csv('/content/combined_data.csv')
#Convert text to a list
text = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()
labels

In [72]:
#Split into train and test sequence
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42)
# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(y_train)
testing_labels_final = np.array(y_test)

In [65]:
from pandas._libs.algos import pad_2d_inplace
#Tokenize the data
tokenizer=Tokenizer(oov_token='<OOV>',num_words=1000)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
x_sequence = tokenizer.texts_to_sequences(X_train)
train_padded= pad_sequences(x_sequence,maxlen=100, padding= 'post')

y_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(y_sequences,maxlen=100, 
                               padding='post', truncating='post')

In [67]:
#Review a Sequence
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[2]))
print(X_train[2])

definitely not worth the 3 i paid ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Definitely not worth the $3 I paid.


In [68]:
## Train a Basic Sentiment Model with Embeddings
model= tf.keras.Sequential([
    tf.keras.layers.Embedding(1000,16,input_length=100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='Accuracy')
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 100, 16)           16000     
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_20 (Dense)            (None, 16)                25616     
                                                                 
 dense_21 (Dense)            (None, 1)                 17        
                                                                 
Total params: 41,633
Trainable params: 41,633
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.fit(train_padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
50/50 [==============================] - 1s 8ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 2/10
50/50 [==============================] - 0s 4ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 3/10
50/50 [==============================] - 0s 4ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 4/10
50/50 [==============================] - 0s 4ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 5/10
50/50 [==============================] - 0s 5ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 6/10
50/50 [==============================] - 0s 4ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 7/10
50/50 [==============================] - 0s 4ms/step - loss: 7.8432 - Accuracy: 0.4915 - val_loss: 7.1906 - val_Accuracy: 0.5338
Epoch 8/10
50/50 [==

In [75]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [77]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, 1000):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [78]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>